In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install streamlit-folium

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 27.0 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from transformers import AutoTokenizer, ElectraModel
from konlpy.tag import Okt
import re
import time

# Kakao Maps API 키 설정
KAKAO_MAP_API_KEY = '<카카오 api 키를 입력하세요>'

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 클래스 정의
class MultiTaskElectraModel(nn.Module):
    def __init__(self, num_type_labels, num_disaster_labels):
        super(MultiTaskElectraModel, self).__init__()
        self.electra = ElectraModel.from_pretrained('beomi/KcELECTRA-base')
        self.dropout = nn.Dropout(0.1)
        self.classifier_type = nn.Linear(self.electra.config.hidden_size, num_type_labels)
        self.classifier_disaster = nn.Linear(self.electra.config.hidden_size, num_disaster_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits_type = self.classifier_type(pooled_output)
        logits_disaster = self.classifier_disaster(pooled_output)
        return logits_type, logits_disaster

# 모델과 토크나이저 설정
num_type_labels = 4
num_disaster_labels = 16
model = MultiTaskElectraModel(num_type_labels, num_disaster_labels)
model.load_state_dict(torch.load('/content/drive/MyDrive/실전프로젝트1 제출폴더/best_model.pt', map_location=device))
model.to(device)
model.eval()
tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base', use_fast=False)

# 레이블 매핑
type_label_mapping = {0: '구급', 1: '구조', 2: '기타', 3: '화재'}
disaster_label_mapping = {
    0: '기타', 1: '기타구급', 2: '기타구조', 3: '기타화재', 4: '대물사고',
    5: '부상', 6: '사고', 7: '산불', 8: '심정지', 9: '안전사고',
    10: '약물중독', 11: '일반화재', 12: '임산부', 13: '자살',
    14: '질병(중증 외)', 15: '질병(중증)'
}

# 위치 정보 추출 함수 정의
def extract_location_with_regex(text):
    pattern = r'([가-힣]+[특별시|광역시|도]\s?[가-힣]+[구|군|시]\s?[가-힣0-9\-]+[로|길|가]?\s?[0-9\-]*[번지]?)'
    matches = re.findall(pattern, text)
    if matches:
        return matches[0]
    else:
        return None

def extract_location_with_okt(text):
    okt = Okt()
    nouns = okt.nouns(text)
    location_candidates = [noun for noun in nouns if len(noun) > 1]

    for loc in location_candidates:
        # 실제 위치 좌표 확인 로직이 필요하다면 이곳에 추가
        return loc  # 위치명만 반환
    return None

# 통합 위치 추출 함수
def extract_location(text):
    location = extract_location_with_regex(text)
    if location:
        return location

    location = extract_location_with_okt(text)
    if location:
        return location

    return None

# 키워드 추출 함수
def extract_keywords(text, num_keywords=10):
    okt = Okt()
    nouns = okt.nouns(text)
    nouns = [n for n in nouns if len(n) > 1]
    freq = {}
    for noun in nouns:
        freq[noun] = freq.get(noun, 0) + 1
    sorted_keywords = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    keywords = [word for word, count in sorted_keywords[:num_keywords]]
    return keywords

# 예측 함수
def predict(text, model, tokenizer, device):
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding=True,
        max_length=256
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        logits_type, logits_disaster = model(input_ids, attention_mask)

    pred_type_idx = logits_type.argmax(dim=1).item()
    pred_disaster_idx = logits_disaster.argmax(dim=1).item()

    return pred_type_idx, pred_disaster_idx

# 텍스트 분석 함수
def analyze_emergency_text(text):
    pred_type_idx, pred_disaster_idx = predict(text, model, tokenizer, device)
    pred_type = type_label_mapping.get(pred_type_idx, "Unknown")
    pred_disaster = disaster_label_mapping.get(pred_disaster_idx, "Unknown")
    keywords = extract_keywords(text)
    location = extract_location(text)
    return pred_type, pred_disaster, keywords, location

# Streamlit UI 설정
st.set_page_config(page_title="AI 응급 상황 분류 시스템", layout="wide")

# 메인 제목
st.title("🚨 AI 기반 응급 상황 분류 시스템")
st.write("대화를 분석하여 응급 상황 유형, 재난 상황, 위치 정보를 추출합니다.")

# 기본 텍스트 설정
dialogue_text = [
    "[SPEAKER_0] 119입니다. 무엇을 도와드릴까요?",
    "[SPEAKER_1] 네, 친구가 등산 중에 사고를 당했어요! 바위에서 미끄러져 크게 다쳤습니다.",
    "[SPEAKER_0] 알겠습니다. 현재 위치가 어디신가요?",
    "[SPEAKER_1] 서울특별시 종로구 청운동 부근의 산 중턱입니다. 위치가 정확하지 않아서요.",
    "[SPEAKER_0] 괜찮습니다. 위치 추적을 시도해보겠습니다. 친구분의 상태를 좀 더 자세히 말씀해주시겠어요?",
    "[SPEAKER_1] 네, 머리를 다쳤고 피가 많이 나고 있어요. 의식은 있지만, 많이 아파하고 움직일 수가 없어요.",
    "[SPEAKER_0] 알겠습니다. 지금 바로 구조대를 보내드리겠습니다. 혹시 응급 처치를 시도해보신 적 있으신가요?",
    "[SPEAKER_1] 아니요, 어떻게 해야 할지 몰라서요.",
    "[SPEAKER_0] 괜찮습니다. 제가 안내해드리겠습니다. 우선 친구분이 가능한 편안한 자세를 취할 수 있도록 해주시고, 머리 부분에 지혈을 위해 깨끗한 천을 대주세요.",
    "[SPEAKER_1] 네, 지금 그렇게 하고 있습니다. 그런데 친구가 많이 불안해하고 있어요.",
    "[SPEAKER_0] 네, 친구분을 안심시켜 주시고, 구급대가 도착할 때까지 지혈을 계속해주세요. 친구분이 너무 움직이지 않도록 주의해주세요.",
    "[SPEAKER_1] 알겠습니다. 구급대가 얼마나 걸릴까요?",
    "[SPEAKER_0] 현재 구급대가 출발한 상태이며 최대한 빠르게 도착할 수 있도록 하겠습니다. 계속해서 친구분을 진정시키며 상태를 지켜봐 주세요."
]

# 세션 상태 초기화
if "current_line" not in st.session_state:
    st.session_state.current_line = 0
    st.session_state.chat_history = []
    st.session_state.pred_type = "Unknown"
    st.session_state.pred_disaster = "Unknown"
    st.session_state.keywords = []
    st.session_state.location = None

# 대화 창을 위한 빈 공간을 준비합니다.
chat_display = st.empty()

# 대화 표시 및 모든 대화 출력 후 분석
for line_index in range(st.session_state.current_line, len(dialogue_text)):
    current_text = dialogue_text[line_index]
    st.session_state.chat_history.append(current_text)
    st.session_state.current_line += 1

    # 채팅 형태로 대화 표시
    with chat_display.container():
        st.write("### 📞 대화 내용")
        for line in st.session_state.chat_history:
            if "[SPEAKER_0]" in line:
                st.markdown(f"<div style='text-align: left; color: blue;'>{line}</div>", unsafe_allow_html=True)
            elif "[SPEAKER_1]" in line:
                st.markdown(f"<div style='text-align: right; color: green;'>{line}</div>", unsafe_allow_html=True)

    # 1초 대기 후 다음 줄로 진행
    time.sleep(1)

# 모든 대화 출력 후 분석 수행
cumulative_text = '\n'.join(st.session_state.chat_history)
pred_type, pred_disaster, keywords, location = analyze_emergency_text(cumulative_text)
st.session_state.pred_type = pred_type
st.session_state.pred_disaster = pred_disaster
st.session_state.keywords = keywords
if location:
    st.session_state.location = location  # 위치 정보 업데이트

# 예측 결과 표시
st.write("### 2️⃣ 분석 결과")
st.subheader("📌 재난 유형 (Type)")
st.write(st.session_state.pred_type)
st.subheader("📌 재난 상황 (Disaster)")
st.write(st.session_state.pred_disaster)
st.subheader("📌 응급 키워드")
st.write(", ".join(st.session_state.keywords) if st.session_state.keywords else "키워드가 추출되지 않았습니다.")

# Kakao Map 표시
if st.session_state.location:
    st.subheader("📌 위치 정보")
    st.write(st.session_state.location)
    kakao_map_url = f"https://map.kakao.com/link/search/{st.session_state.location}"
    st.markdown(
        f'<iframe width="100%" height="400" src="{kakao_map_url}"></iframe>',
        unsafe_allow_html=True
    )
else:
    st.write("위치 정보가 추출되지 않았습니다.")

Overwriting app.py


In [ ]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
!curl https://loca.lt/mytunnelpassword

34.139.107.141

In [ ]:
!streamlit run app.py --server.address=0.0.0.0 --server.port=8501 --server.headless=true &>/content/logs.txt & npx localtunnel --port 8501

^C
